In [ ]:
##  #READ ME ####

# The folder contain 172 genome sequence files, named GCF_*.fna , each .fna file is a genome, may contain one or multiple sequences.
# File GCF_000001405.40 is the human genome, make sure to include this sequence in your training.
# For training, you can use the code below to first truncate the sequences into the length neede, for example, 8,000 basepairs as showed below. 
# Please keep all truncated sequneces from the human genome GCF_000001405.40, then down-sample the rest according to your need for training.You can perform dedup before down-sampling. 

In [ ]:
## split genome into specific length


import os  
from concurrent.futures import ProcessPoolExecutor  
from Bio import SeqIO  
  
input_dir = "/home/chuancao/genome/refseq_genome"  # Replace with the path to your .fna files  
output_file = "/home/chuancao/dna/refseq_genome_8k.fasta"  # Replace with the path to your output .fasta file  
chunk_size = 8000  # Size of the sequence chunks  
num_workers = 64  # Number of worker processes  
  
def process_file(file_path):  
    # This function will be executed by each worker process  
    chunks = []  
    with open(file_path, "r") as handle:  
        for record in SeqIO.parse(handle, "fasta"):  
            seq = str(record.seq)  
            for i in range(0, len(seq), chunk_size):  
                chunk_seq = seq[i:i + chunk_size]  
                # Make sure we don't get sequences smaller than the chunk size  
                if len(chunk_seq) == chunk_size:  
                    # Create a new SeqRecord  
                    chunk_record = SeqIO.SeqRecord(Seq(chunk_seq), id=f"{record.id}_{i//chunk_size}", description="")  
                    chunks.append(chunk_record)  
    return chunks  
  
def save_chunks_to_file(chunks, output_file):  
    with open(output_file, "a") as output_handle:  # Open file in append mode  
        for chunk in chunks:  
            SeqIO.write(chunk, output_handle, "fasta")  
  
# List all .fna files in the input directory  
fna_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.fna')]  
  
# Process the files in parallel using ProcessPoolExecutor  
with ProcessPoolExecutor(max_workers=num_workers) as executor:  
    # Use a dictionary to keep track of running tasks  
    future_to_file = {executor.submit(process_file, fna_file): fna_file for fna_file in fna_files}  
      
    # As tasks complete, save the resulting chunks to the output file  
    for future in concurrent.futures.as_completed(future_to_file):  
        chunks = future.result()  
        save_chunks_to_file(chunks, output_file)  
  
print("Splitting complete.")  
